In [71]:
import requests
import json

In [45]:
def get_player_ids_by_country(country):
    #use the country's 2-character ISO 3166 code (capitalized) to specify which country you want data for
    #example: United States = US
    url = f'https://api.chess.com/pub/country/{country}/players'
    response = requests.get(url)
    
    #check if the request worked
    assert response.status_code == 200 
    
    #grab pgn text
    names = str(response.content).split(',')
    
    return [name[1:-2] for name in names][1:]

['__novice_',
 '_-jerrod-',
 '_0x4',
 '_agentcooper',
 '_ai',
 '_crazyfo',
 '_foreg',
 '_hrakkar',
 '_jon',
 '_justme',
 '_killer_kwee',
 '_klk',
 '_krypt0',
 '_ks',
 '_larax',
 '_philosophe',
 '_pianoking',
 '_razo',
 '_scaramouche',
 '_scylla',
 '_solsky',
 '_swag',
 '_vivan',
 '_y',
 '_zer',
 '----andy----',
 '-cerealkiller',
 '-cpm',
 '-devioustyles211',
 '-eli',
 '-h-a-t-r-e-d',
 '-jarod',
 '-jg',
 '-kingofthecastle',
 '-kingsrook',
 '-mega',
 '-not-sure',
 '-osmium',
 '-redak',
 '-rookie',
 '-shadowfox',
 '-vitovitkauska',
 '0_0-0_0-0_',
 '0_greenpiece_',
 '0_ota',
 '0-n-',
 '0-o_',
 '00_chess_dictator_0',
 '00000011',
 '000011111',
 '0005160',
 '0007373',
 '000jr00',
 '000t',
 '001010',
 '001010111001',
 '001431',
 '0017th50',
 '006n',
 '007adam',
 '007blac',
 '007devi',
 '007ell',
 '007',
 '007jl',
 '007n',
 '007',
 '007safecracke',
 '008922',
 '008kill',
 '0091',
 '0094262',
 '00arch_bishop0',
 '00checkmat',
 '00chessg',
 '00dave',
 '00dle',
 '00duc',
 '00edga',
 '00heat',
 '0

In [ ]:
pick N random player names

#january 1st 2016 in epoch time
t = 1451606400

#number of player ids to retrieve 
num_of_players = 30000

players = []

while len(players) < num_of_players:
    if (date_joined(username) > t) and (games_played(username) > 10):
        players.append(username)


In [50]:
is_profile_created_after('_jon', 0)

<Response [404]>

In [73]:
def date_joined(username):
    response = requests.get(f'https://api.chess.com/pub/player/{username}')
    profile = json.loads(response.content.decode('utf-8'))
    return profile['joined']


1424034189